*Coherence Calculations*


For coherence calculation we need: Import all docs, get data from DB, wiki vocab and dict, stop words


Then sample from db, prepare corpus, construct doc term matrix, THEN compute coherence values and plot coherence plots, THEN run LDA-> then plot based on that ...

In [1]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim import similarities
#ssh -L 3306:127.0.0.1:3306 -N -f lem325@eltanin.dept.lehigh.edu

In [2]:
# Importing the libiraries
import gensim

#check Gensim version
assert '3.8.2' <= gensim.__version__ <= '3.8.3', 'You must install Gensim 3.8.3 or 3.8.2 to be able to run LDA Mallet'

from gensim.models.wrappers import LdaMallet
from gensim.models.coherencemodel import CoherenceModel
from gensim.models import LdaModel
import pandas as pd
import numpy as np
import math
import sys
import matplotlib.pyplot as plt
import pickle
from scipy.spatial.distance import jensenshannon

import platform #May not be needed but alas
#checking OS
if 'windows' in platform.system().lower():
	mallet_path = 'C:/mallet-2.0.8/bin/mallet'
elif 'linux' in platform.system().lower():
    import os
    os.environ['MALLET_HOME'] = 'mallet-2.0.8/'
    mallet_path = 'mallet-2.0.8/bin/mallet' # you should NOT need to change this
#     os.environ['MALLET_HOME'] = '../mallet-2.0.8/'
#     mallet_path = '../mallet-2.0.8/bin/mallet' # you should NOT need to change this 
else:
	mallet_path = 'mallet'

In [3]:
#pre-processing 
#this part uses Amin's code 
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim import similarities



import csv, sys
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet


import pandas as pd
import numpy as np

import copy


try:
	csv.field_size_limit(sys.maxsize)
except:
	print('Error in setting maxSize for CSV output')
    
def preprocess_data(doc_list, id_list, extra_stopwords = {},len_th=4,lemmatized=False):
	'''
	Returns: a list of process dataset and origianl documents of those documents

	This function removes stop-wrods, lemmatized the documens, if stated, and eliminates the documnets 
	with lenhgth of 4 or less. 
	***These processes may result in lower number of documents than the original number. To make sure 
	you receive both the original docs and the processed doc in similar order we return both.

	parameter doc_list: a list of string (documents)
	parameter extra_stopwords: NLTK.stop_words are used, if you wish to add to that list, you can use yours.
	parameter len_th: documents with len_th and less will be removed.
	parameter lemmatized: If true, the terms will be lemmatized. **be aware that lemmatization of the documents
	will result in different topics and may need different evaluation, including NPMI, stability, or human assessment**

	'''

	# replace single smart quote with single straight quote, so as to catch stopword contractions
	doc_list = [re.sub("[\u2018\u2019]", "'", doc) for doc in doc_list]
	doc_list = [re.sub('\d+', '', doc) for doc in doc_list]
	doc_list = [re.sub('(\/.*?\.[\w:]+)', '', doc) for doc in doc_list]
	#doc_list = [re.sub('pdf|icon|jpg', '', doc) for doc in doc_list]
	#doc_list = [re.sub('(http(s)?:\/\/.)?(www\.)?[-a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)', '', doc) for doc in doc_list]
	doc_list = [re.sub(r"http\S+", '', doc) for doc in doc_list]

	# initialize regex tokenizer
	tokenizer = RegexpTokenizer(r'\w+')
	# create English stop words list
	en_stop = set(stopwords.words('english'))
	# add any extra stopwords
	if (len(extra_stopwords) > 0):
		en_stop = en_stop.union(extra_stopwords)

	#defining a lemmatizer
	lemmatizer = WordNetLemmatizer()

	# list for tokenized documents in loop
	texts = []
	original_docs = []
	# loop through document list
	c = 0 #counter on the document number
	for i in doc_list:
		# clean and tokenize document string
		raw = i.lower()
		tokens = tokenizer.tokenize(raw)
		stopped_tokens = []
		# remove stop words from tokens
		#stopped_tokens = [i for i in tokens if not i in en_stop and len(i)>1]
		if lemmatized:
		  for t in tokens:
		    if t not in en_stop and len(t)>1:
		      pos=nltk_tag_to_wordnet_tag(nltk.pos_tag([t])[0][1])
		      if pos:
		        stopped_tokens.append(lemmatizer.lemmatize(t,pos=pos))
		      else:
		        stopped_tokens.append(lemmatizer.lemmatize(t))
		  #     print(t,pos,nltk.pos_tag([t])[0][1])
		  # print(stopped_tokens)
		  #stopped_tokens = [lemmatizer.lemmatize(i,pos=nltk_tag_to_wordnet_tag(nltk.pos_tag([i])[0][1])) for i in tokens if not i in en_stop and len(i)>1]
		else:
		  stopped_tokens = [i for i in tokens if not i in en_stop and len(i)>1]


		# add tokens to list
		if len(stopped_tokens) >= len_th:
		  texts.append([stopped_tokens, i, id_list[c]]) # [pre-processed text (tokenized), original, id]
		  #original_docs.append([i,c])

		c += 1

	return texts

[nltk_data] Downloading package stopwords to /home/lem325/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/lem325/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/lem325/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/lem325/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Set the stop words

In [4]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim import similarities
#ssh -L 3306:127.0.0.1:3306 -N -f lem325@eltanin.dept.lehigh.edu

In [5]:
import pickle


#read all the stop words and add them to the list of extra stop words..
extra_stop_words = open('Stopword_list.txt', 'r')
#addiong some stop words...
extra_stop_words_list = extra_stop_words.readlines()
extra_stopwords = set()
for item in extra_stop_words_list:
    extra_stopwords.add(item.strip())
extra_stopwords.add('amp');extra_stopwords.add('&amp');extra_stopwords.add('&amp;')

# print(extra_stopwords)


Load the wiki vocab and dict for coherence and update stop words

In [6]:
with open('wiki_vocab_dict','rb') as read_file:
    wiki_vocab_dict = pickle.load(read_file)

with open('vocab_dict', 'rb') as read_file:
    vocab_dict = pickle.load(read_file)

In [7]:
# set(vocab_dict.token2id.keys())
extra_stopwords = extra_stopwords.union(set(vocab_dict.token2id.keys()).difference(set(wiki_vocab_dict.token2id.keys())))

Import docs aka access database and preprocess

In [9]:
# Put code to grab from db here:
#ssh -L 3306:127.0.0.1:3306 -N -f lem325@eltanin.dept.lehigh.edu

#Read orig text and preproc from SQL db
#CONNECT TO SQL
#import mysql

import pymysql
pymysql.install_as_MySQLdb()
import MySQLdb

sql_db = MySQLdb.connect(host='127.0.0.1',user='covidAnalysis', password = "k34p63MbDDcZ9yf4",db='covid19framing')
cursor = sql_db.cursor()

#originaltext = [item[1].decode() for item in results]




In [10]:
cursor.execute("select * from articles where text is not NULL")
results = cursor.fetchall()
len(results)




3800

In [11]:
from random import sample
import random
#SUBSAMPLE OF 5 percent
random.seed(123456)
sample_size = int(len(results)*.02)
completedocssampled = sample(results,sample_size)

In [ ]:
doclist[5]

In [12]:
doclist = [item[-1].split() for item in completedocssampled] #preprocessed 
originaltext = [item[1] for item in completedocssampled] #originaltext

In [34]:
del completedocs

NameError: name 'completedocs' is not defined

In [13]:
#saving the sample documents:
#Lowkey optional I think
with open('completedocssampled.obj', 'wb') as objwriter:
    pickle.dump(completedocssampled, objwriter)

In [14]:
def prepare_corpus(doc_clean,no_below=5,no_above=0.5):
	'''
	Reutrns: A dictionary of the final set of terms and document-term frequency matrix

	# adapted from https://www.datacamp.com/community/tutorials/discovering-hidden-topics-python

	parameter doc_clean: processed set of documents (type: string)
	parameter no_below: exclude words that only appear $no_below$ times or less in the whole corpus
	parameter no_above: any words included in more than $no_above$ percentage of documents will be excluded

	'''
	# Creating the term dictionary of our courpus, where every unique term is assigned an index.
	dictionary = corpora.Dictionary(doc_clean)

	dictionary.filter_extremes(no_below=no_below, no_above=no_above)
	# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
	doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
	# generate LDA model
	return dictionary,doc_term_matrix


In [15]:
def prepare_corpus(doc_clean,no_below=5,no_above=0.5):
	'''
	Reutrns: A dictionary of the final set of terms and document-term frequency matrix

	# adapted from https://www.datacamp.com/community/tutorials/discovering-hidden-topics-python

	parameter doc_clean: processed set of documents (type: string)
	parameter no_below: exclude words that only appear $no_below$ times or less in the whole corpus
	parameter no_above: any words included in more than $no_above$ percentage of documents will be excluded

	'''
	# Creating the term dictionary of our courpus, where every unique term is assigned an index.
	dictionary = corpora.Dictionary(doc_clean)

	dictionary.filter_extremes(no_below=no_below, no_above=no_above)
	# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
	doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
	# generate LDA model
	return dictionary,doc_term_matrix


In [16]:
#Actually calling prepare corpus here


# if you need to subsample, you can subsample here:
#N = int(len(doclist)/1000)
vocab_dict, doc_term_matrix = prepare_corpus(doclist)

In [17]:
def compute_coherence_values(dictionary, corpus, texts, ref_dict=[], limit=25, start=5, step=5,threshold=0.10,runs = 1):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    purity_values: Average purity for each run
    contrast_values: Average of contrast for each run
    df: DataFrame df inlcudes all results and number of topics associated with those results
    """
    coherence_values = []
    coherence_std = []
    model_list = []
    purity_values = []
    contrast_values = []
    all_top_terms = []#this list will store all topics of all models as a list of top terms.
    # so with two models one with 10 and one with 20, we will have 30 of top terms
    top_terms_count_ls = [0] #this list keeps track of topics we add in each run and for each num_topics
    df = pd.DataFrame(columns=['num_topics','coherence','purity','contrast'])
    for num_topics in range(start, limit+1, step):
      model_t = []
      purity_t = []
      coherence_t = []
      contrast_t = []
      for r in range(runs):
          #model=LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics)
          model = LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=dictionary,optimize_interval = 25,iterations=2000, workers = 6) # only add it on server and not local 
          #model_t.append(model)

          #storing top_terms
          for tn in range(num_topics): 
          	tt = model.show_topic(tn,topn=20)
          	#saving top_terms and their counts
          	all_top_terms.append([i[0] for i in tt])
				  #saving counts of top_terms
          top_terms_count_ls.append(len(all_top_terms))

          #purity computation
#           topic_term_cond = get_conditional_probabilities(model,num_topics)
          topic_term = model.get_topics()#model.show_topics(num_topics = num_topics, num_words=200)
          #go over each topic and compute set of words that satisfy p(t|w)>=threshold
          pur = []
          cont = []
          for t in range(num_topics):
            pur.append(0.0)
            cont.append(0.0)
#             w_ind = np.argwhere(topic_term_cond[t,:]>=threshold)
#             pur.append(np.sum(topic_term[t,w_ind]))
#             cont.append(np.mean(topic_term_cond[t,w_ind]))
          
          #for n in range(num_topics):
            #pur.append(sum([float(i) for i in re.findall(r'\d*\.\d+',topic_term[n][1]) if float(i)>=threshold]))

          purity_t.append(np.mean(pur))
          contrast_t.append(np.mean(cont))
          # coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_npmi',processes=1)
          # coherence_value = coherencemodel.get_coherence()
          # coherence_t.append(coherence_value)
          df = df.append({'num_topics':num_topics,'purity':np.mean(pur),'contrast':np.mean(cont)},ignore_index=True)

        #stroing the results
#       model_list.append(model)
      purity_values.append(purity_t)
      #coherence_values.append(coherence_t)
      contrast_values.append(contrast_t)
      print('{0} number of topics has been processed'.format(num_topics))

    #computing coherence for all top terms at once
    cscore = CoherenceModel(topics=all_top_terms,dictionary=ref_dict,texts=texts,coherence='c_npmi',processes=7).get_coherence_per_topic() #processes are the number of threads in computing coherence

    for i in range(0,len(top_terms_count_ls)-1):
    	coherence_values.append(np.mean(cscore[top_terms_count_ls[i]:top_terms_count_ls[i+1]]))
    	coherence_std.append(np.std(cscore[top_terms_count_ls[i]:top_terms_count_ls[i+1]]))
    	#update dataframe
    df.coherence = coherence_values
    df['coherence_std'] = coherence_std
    return model_list, coherence_values, purity_values, contrast_values,df

In [18]:
 #2.1 function to load wikipedia corpus
def loading_wiki_docs(filename:str):

    # Returns: documents of wikipedia corpus
    # loads a wikipedia text file and return documents with length>3
    # parameter filename: name of the wikipedia text documents (type:str)

    wiki_docs = []

    with open(filename,'r',encoding="utf-8") as f:
        d = f.readline()
        wiki_docs.append(d)
    while(d):
        d = f.readline()
        if len(d) > 3:
            wiki_docs.append(d)
    return wiki_docs

In [19]:
#2.2 load wikipedia for coherence computing
#topic coherence
wiki_docs = loading_wiki_docs('../wiki_corpus/wiki_full')
#doing pre-processing on wiki-pedia documents
pre_processed_wiki = preprocess_data(wiki_docs,[0]*len(wiki_docs))
pre_processed_wiki = [i[0] for i in pre_processed_wiki]
wiki_vocab_dict, _ = prepare_corpus(pre_processed_wiki)
del wiki_docs

FileNotFoundError: [Errno 2] No such file or directory: '../wiki_corpus/wiki_full'

In [ ]:
# import pickle

# with open('pre_processed_wiki','wb') as save_file:
#     pickle.dump(pre_processed_wiki,save_file)
        
# with open('wiki_vocab_dict','wb') as save_file:
#     pickle.dump(wiki_vocab_dict,save_file)

In [ ]:
# impodoc_term_matrixe
with open('pre_processed_wiki','rb') as read_file:
    pre_processed_wiki = pickle.load(read_file)
        
with open('wiki_vocab_dict','rb') as read_file:
    wiki_vocab_dict = pickle.load(read_file)

#with open('doc_term_matrix', 'rb') as read_file:
    #doc_term_matrix = pickle.load(read_file)

with open('vocab_dict', 'rb') as read_file:
    vocab_dict = pickle.load(read_file)

In [ ]:
#subsample wikipedia dictionary
from random import sample
import random
random.seed(123456)
wiki_sample_size = int(len(pre_processed_wiki)*.05)

pre_processed_wiki = sample(pre_processed_wiki,wiki_sample_size)


In [29]:
len(pre_processed_wiki)

265112

In [31]:
import seaborn as sns

ImportError: /share/Apps/anaconda3/2020.07/envs/nlp/lib/python3.8/site-packages/scipy/cluster/_vq.cpython-38-x86_64-linux-gnu.so: failed to map segment from shared object

In [ ]:
import seaborn as sns
def plotting_coherence(eval_df):
    ax =sns.pointplot(x='num_topics', y='coherence', data=eval_df)
    plt.title('Coherence score with Wiki docs as ref corpus')
    plt.show()
    ax = plt.errorbar(x='num_topics', y='coherence', yerr='coherence_std', data=eval_df[0:-1:3])
    plt.title('Coherence score with std within a single run')
    plt.show

In [36]:
#3.run the LDA for the nuber of topics documents found above
from random import sample
import random
random.seed(123456)
lim = 40
st = 40
stp = 2
n_runs = 2
sample_size = 1000000
models, coherence, pur, cont,eval_df = compute_coherence_values(dictionary=vocab_dict, corpus=doc_term_matrix, texts=pre_processed_wiki,ref_dict=wiki_vocab_dict, limit=lim, start=st, step=stp,threshold=0.10,runs = n_runs)
#running on a VM machine
# eval_df.to_csv('coherence_ap_10to90.csv',index=False)
# plotting_coherence(eval_df)
eval_df.to_csv('coherence_{0}to{1}_st{2}.csv'.format(st,lim,stp))

MemoryError: 

In [33]:
eval_df

NameError: name 'eval_df' is not defined

In [ ]:
plotting_coherence(eval_df)

In [ ]:
#merge the ranges of coherence.

coherence_10_90 = pd.read_csv('coherence_10to20_st20.csv')
coherence_10_90 = coherence_10_90.append(pd.read_csv('coherence_30to50_st20.csv'))
coherence_10_90 = coherence_10_90.append(pd.read_csv('coherence_35to45_st10.csv'))
coherence_10_90 = coherence_10_90.append(pd.read_csv('coherence_55to55_st10.csv'))
coherence_10_90 = coherence_10_90.append(pd.read_csv('coherence_70to90_st20.csv'))
coherence_10_90 = coherence_10_90.append(pd.read_csv('coherence_36to54_st4.csv'))
coherence_10_90 = coherence_10_90.append(pd.read_csv('coherence_37to43_st2.csv'))
coherence_10_90 = coherence_10_90.append(pd.read_csv('coherence_38to38_st2.csv'))

In [ ]:
plotting_coherence(coherence_10_90)

In [ ]:
coherence_10_90